In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
current_dir = os.getcwd()
libs_path = os.path.abspath(os.path.join(current_dir, "..", "libs"))
if libs_path not in sys.path:
    sys.path.append(libs_path)

In [2]:
import pandas as pd
from IPython.display import display
from ipywidgets import interact
import numpy as np
pd.set_option('display.precision', 3)
import data_processing as dp

In [20]:
res=dp.load_datasets()

SelectMultiple(description='Датасеты:', layout=Layout(width='500px'), options={'.ipynb_checkpoints': 'E:\\stud…

Button(description='ОК', style=ButtonStyle())

Output()

In [21]:

for fn in res.options:
    if res.options[fn] not in res.value:
        continue
    df = dp.Dataset(pd.read_excel(res.options[fn]), name=fn)



In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
from dataclasses import dataclass, field
from typing import Optional, Tuple, Dict, Any
from sklearn.preprocessing import StandardScaler
import math

# ---------------------------------------------------------
# 1. Конфигурация (Dataclass)
# ---------------------------------------------------------

@dataclass
class ModelConfig:
    """Конфигурация модели и обучения."""
    # Данные
    window_size: int = 10
    num_nodes: int = 5  # Количество признаков (колонок)
    
    # Архитектура
    input_dim: int = 1
    embed_dim: int = 32       # Размерность эмбеддинга
    hidden_dim: int = 64      # Размерность скрытых слоев
    latent_dim: int = 16      # Размерность латентного вектора
    num_heads: int = 4        # Количество голов внимания
    num_layers: int = 2       # Количество слоев энкодера/декодера
    dropout: float = 0.2
    
    # Граф
    use_learnable_graph: bool = True
    graph_reg_lambda: float = 1e-4  # Коэффициент регуляризации графа (спарсность)
    
    # Обучение
    learning_rate: float = 1e-3
    weight_decay: float = 1e-5
    device: str = "auto"      # "cuda", "cpu" или "auto"

# ---------------------------------------------------------
# 2. Слои Архитектуры
# ---------------------------------------------------------

class PositionalEncoding(nn.Module):
    """Позиционное кодирование для временных шагов."""
    def __init__(self, d_model: int, max_len: int = 5000, dropout: float = 0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0) # (1, max_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (Batch, Time, Nodes, Dim) -> нужно применить по времени
        # Для упрощения применим к измерению Time, предполагая, что Nodes независимы в позиции
        # Транспонируем для совместимости с классическим PE
        b, t, n, d = x.shape
        x = x + self.pe[:, :t, :d].unsqueeze(2) # (B, T, 1, D) broadcast to (B, T, N, D)
        return self.dropout(x)







# ---------------------------------------------------------
# Исправленные слои Архитектуры
# ---------------------------------------------------------

class TemporalSelfAttention(nn.Module):
    """Внимание по временной оси (Multi-Head)."""
    def __init__(self, dim: int, num_heads: int, dropout: float = 0.1):
        super().__init__()
        self.attention = nn.MultiheadAttention(embed_dim=dim, num_heads=num_heads, dropout=dropout, batch_first=True)
        self.norm = nn.LayerNorm(dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (Batch, Time, Nodes, Dim)
        b, t, n, d = x.shape
        # Переставляем: (Batch * Nodes, Time, Dim)
        # .contiguous() важен после permute
        x = x.permute(0, 2, 1, 3).contiguous().view(-1, t, d)
        
        attn_out, _ = self.attention(x, x, x)
        x = x + self.dropout(attn_out)
        x = self.norm(x)
        
        # Возвращаем форму: (Batch, Time, Nodes, Dim)
        # .contiguous() перед view для безопасности
        x = x.view(b, n, t, d).permute(0, 2, 1, 3).contiguous()
        return x

class SpatialGraphAttention(nn.Module):
    """Внимание по оси признаков (Графовый слой с обучаемой смежностью)."""
    def __init__(self, dim: int, num_nodes: int, dropout: float = 0.1, use_learnable: bool = True):
        super().__init__()
        self.num_nodes = num_nodes
        self.use_learnable = use_learnable
        
        if self.use_learnable:
            # Обучаемые эмбеддинги узлов для вычисления внимания
            self.node_embeddings = nn.Parameter(torch.Tensor(num_nodes, dim))
            nn.init.xavier_uniform_(self.node_embeddings)
        
        self.w_q = nn.Linear(dim, dim)
        self.w_k = nn.Linear(dim, dim)
        self.w_v = nn.Linear(dim, dim)
        
        self.norm = nn.LayerNorm(dim)
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.ReLU()

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        # x: (Batch, Time, Nodes, Dim)
        b, t, n, d = x.shape
        
        # Вычисляем матрицу внимания A (N, N)
        if self.use_learnable:
            # Attention Score = Softmax(Emb * Emb^T)
            adj = torch.matmul(self.node_embeddings, self.node_embeddings.T)
            adj = F.softmax(adj, dim=1)
        else:
            # Полносвязный граф (все со всеми)
            adj = torch.ones(n, n, device=x.device) / n
            
        adj_matrix = adj # Для регуляризации
        
        # Применяем внимание к признакам
        # x: (B, T, N, D) -> (B*T, N, D)
        # ИСПРАВЛЕНИЕ: используем reshape вместо view для неконтинуальных тензоров
        x_flat = x.reshape(-1, n, d)
        
        Q = self.w_q(x_flat)
        K = self.w_k(x_flat)
        V = self.w_v(x_flat)
        
        # Graph Attention: A * V
        # Нужно расширить adj до батча
        adj_batch = adj.unsqueeze(0).expand(b * t, -1, -1)
        
        out = torch.matmul(adj_batch, V)
        out = self.activation(out)
        out = self.dropout(out)
        
        # Residual
        out = out + x_flat
        out = self.norm(out)
        
        # ИСПРАВЛЕНИЕ: reshape при возврате формы
        return out.reshape(b, t, n, d), adj_matrix

# ---------------------------------------------------------
# Исправленная Основная Модель (Decoder Fix)
# ---------------------------------------------------------

class SpatioTemporalGAE(nn.Module):
    def __init__(self, config: ModelConfig):
        super().__init__()
        self.config = config
        self.num_nodes = config.num_nodes
        self.window_size = config.window_size
        
        # Input Projection
        self.input_proj = nn.Linear(config.input_dim, config.embed_dim)
        self.pos_encoder = PositionalEncoding(config.embed_dim, config.window_size, config.dropout)
        
        # Encoder Stack
        self.encoder_layers = nn.ModuleList([
            EncoderBlock(config.embed_dim, config.num_nodes, config.num_heads, config.dropout, config.use_learnable_graph)
            for _ in range(config.num_layers)
        ])
        
        # Bottleneck (Latent Space)
        self.bottleneck = nn.Sequential(
            nn.Linear(config.embed_dim * config.num_nodes, config.hidden_dim),
            nn.ReLU(),
            nn.Dropout(config.dropout),
            nn.Linear(config.hidden_dim, config.latent_dim)
        )
        
        # Decoder Stack (Symmetric)
        self.decoder_proj = nn.Linear(config.latent_dim, config.hidden_dim)
        self.decoder_layers = nn.ModuleList([
            EncoderBlock(config.embed_dim, config.num_nodes, config.num_heads, config.dropout, config.use_learnable_graph)
            for _ in range(config.num_layers)
        ])
        
        # Output Projection
        self.output_proj = nn.Linear(config.embed_dim, config.input_dim)
        
        # ИСПРАВЛЕНИЕ: Выносим линейный слой декодера в __init__
        # Ранее он создавался в forward, что ломало граф вычислений и регистр параметров
        self.decoder_expand = nn.Linear(config.hidden_dim, config.embed_dim)

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        x: (Batch, Time, Nodes)
        Returns: reconstruction, latent_vector, graph_loss
        """
        b, t, n = x.shape
        assert n == self.num_nodes, f"Expected {self.num_nodes} nodes, got {n}"
        assert t == self.window_size, f"Expected window {self.window_size}, got {t}"
        
        # 1. Embedding
        x = x.unsqueeze(-1) # (B, T, N, 1)
        x = self.input_proj(x) # (B, T, N, Embed)
        x = self.pos_encoder(x)
        
        # 2. Encoder
        adj_reg_loss = 0.0
        for layer in self.encoder_layers:
            x, adj = layer(x)
            if self.config.use_learnable_graph and self.config.graph_reg_lambda > 0:
                # L1 regularization for sparsity
                adj_reg_loss += torch.norm(adj, 1)
        
        # 3. Latent Space
        # Flatten (N, Embed) -> Vector
        x_flat = x.reshape(b, t, -1) # (B, T, N*Embed)
        # Global Pooling over Time (Mean)
        x_pool = x_flat.mean(dim=1) # (B, N*Embed)
        latent = self.bottleneck(x_pool) # (B, Latent)
        
        # 4. Decoder
        # Expand latent back to (B, T, N, Embed)
        dec_h = self.decoder_proj(latent) # (B, Hidden)
        # Repeat for Time steps
        dec_h = dec_h.unsqueeze(1).repeat(1, t, 1) # (B, T, Hidden)
        # Project to Embed dim and reshape to Nodes
        # ИСПРАВЛЕНИЕ: используем предопределенный слой decoder_expand
        dec_h = dec_h.unsqueeze(2).repeat(1, 1, n, 1) # (B, T, N, Hidden)
        x = self.decoder_expand(dec_h) # (B, T, N, Embed)
        
        # Pass through Decoder Layers
        for layer in self.decoder_layers:
            x, _ = layer(x)
            
        # 5. Output
        recon = self.output_proj(x) # (B, T, N, 1)
        recon = recon.squeeze(-1) # (B, T, N)
        
        return recon, latent, adj_reg_loss













class EncoderBlock(nn.Module):
    def __init__(self, dim: int, num_nodes: int, num_heads: int, dropout: float, use_learnable_graph: bool):
        super().__init__()
        self.temporal_attn = TemporalSelfAttention(dim, num_heads, dropout)
        self.spatial_attn = SpatialGraphAttention(dim, num_nodes, dropout, use_learnable_graph)
        self.ffn = nn.Sequential(
            nn.Linear(dim, dim * 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(dim * 2, dim)
        )
        self.norm_ffn = nn.LayerNorm(dim)

    def forward(self, x: torch.Tensor):
        x = self.temporal_attn(x)
        x_spatial, adj = self.spatial_attn(x)
        
        # FFN
        out = self.ffn(x_spatial)
        out = out + x_spatial
        out = self.norm_ffn(out)
        return out, adj

# ---------------------------------------------------------
# 3. Основная Модель (Factory Pattern)
# ---------------------------------------------------------

def create_advanced_gae(config: ModelConfig) -> Dict[str, Any]:
    """
    Фабричная функция для создания модели, оптимизатора и потерь.
    """
    device = config.device
    if device == "auto":
        device = "cuda" if torch.cuda.is_available() else "cpu"
    
    model = SpatioTemporalGAE(config).to(device)
    
    optimizer = optim.AdamW(
        model.parameters(), 
        lr=config.learning_rate, 
        weight_decay=config.weight_decay
    )
    
    # MSE для реконструкции
    criterion_rec = nn.MSELoss(reduction='none')
    # MSE для латентного пространства (если нужно сравнивать с центроидом)
    criterion_lat = nn.MSELoss()
    
    return {
        "model": model,
        "optimizer": optimizer,
        "criterion_rec": criterion_rec,
        "criterion_lat": criterion_lat,
        "device": device,
        "config": config
    }

# ---------------------------------------------------------
# 4. Тренер и Детектор Аномалий
# ---------------------------------------------------------

class GraphAnomalyDetector:
    def __init__(self, df: pd.DataFrame, config: ModelConfig):
        self.df = df
        self.config = config
        self.config.num_nodes = df.shape[1] # Авто-определение узлов
        self.scaler = StandardScaler()
        self.model_dict = create_advanced_gae(self.config)
        self.train_stats = {}

    def _prepare_data(self, df: pd.DataFrame, fit_scaler: bool = False) -> np.ndarray:
        data = df.values
        if fit_scaler:
            data = self.scaler.fit_transform(data)
        else:
            data = self.scaler.transform(data)
            
        n_samples = data.shape[0] - self.config.window_size + 1
        windows = np.zeros((n_samples, self.config.window_size, data.shape[1]))
        for i in range(n_samples):
            windows[i] = data[i : i + self.config.window_size]
        return windows.astype(np.float32)

    def fit(self, epochs: int = 100, batch_size: int = 64, verbose: bool = True):
        model = self.model_dict["model"]
        optimizer = self.model_dict["optimizer"]
        criterion_rec = self.model_dict["criterion_rec"]
        device = self.model_dict["device"]
        config = self.model_dict["config"]
        
        train_data = self._prepare_data(self.df, fit_scaler=True)
        tensor_data = torch.FloatTensor(train_data).to(device)
        dataset = torch.utils.data.TensorDataset(tensor_data)
        loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
        
        model.train()
        for epoch in range(epochs):
            total_loss = 0
            total_rec_loss = 0
            total_graph_loss = 0
            
            for batch in loader:
                x = batch[0]
                optimizer.zero_grad()
                
                recon, latent, graph_reg = model(x)
                
                # Reconstruction Loss
                rec_loss = criterion_rec(recon, x).mean()
                
                # Total Loss
                loss = rec_loss + (config.graph_reg_lambda * graph_reg)
                
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                
                total_loss += loss.item()
                total_rec_loss += rec_loss.item()
                total_graph_loss += graph_reg.item()
            
            if verbose and (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss/len(loader):.4f} | Rec: {total_rec_loss/len(loader):.4f}")
        
        # Сохраняем статистику обучения для порога
        model.eval()
        with torch.no_grad():
            recon, latent, _ = model(tensor_data)
            errors = criterion_rec(recon, tensor_data).mean(dim=(1, 2)).cpu().numpy()
            self.train_stats['mean_error'] = np.mean(errors)
            self.train_stats['std_error'] = np.std(errors)
            self.train_stats['threshold'] = np.percentile(errors, 95) # Default 95%

    def detect(self, threshold_percentile: float = 95.0) -> pd.DataFrame:
        model = self.model_dict["model"]
        criterion_rec = self.model_dict["criterion_rec"]
        device = self.model_dict["device"]
        
        model.eval()
        data_windows = self._prepare_data(self.df, fit_scaler=False)
        tensor_data = torch.FloatTensor(data_windows).to(device)
        
        with torch.no_grad():
            recon, latent, _ = model(tensor_data)
            
            # 1. Reconstruction Error
            rec_errors = criterion_rec(recon, tensor_data).mean(dim=(1, 2)).cpu().numpy()
            
            # 2. Latent Deviation (Optional robustness check)
            # Можно сравнивать с центроидом латентного пространства обучения
            # Для простоты используем только Rec Error + сглаживание
            
        # Маппинг ошибок обратно на временную ось
        # Окно i покрывает точки [i, i+T-1]. Приписываем ошибку центру окна или концу.
        full_scores = np.zeros(len(self.df))
        mid_point = self.config.window_size // 2
        
        for i, err in enumerate(rec_errors):
            idx = i + mid_point
            if idx < len(full_scores):
                # Накопление ошибок (если точка попадает в несколько окон)
                full_scores[idx] += err
                # В реальной реализации нужно делить на количество покрытий, 
                # но для детекции пиков это не критично.
        
        # Нормализация скоринга (простая)
        # Заполняем края
        full_scores[:mid_point] = full_scores[mid_point]
        full_scores[-mid_point:] = full_scores[-mid_point-1]
        
        # Динамический порог
        threshold = np.percentile(rec_errors, threshold_percentile)
        # Для полного ряда используем статистику обучения, если тестовый похож
        anomaly_mask = full_scores > (self.train_stats['mean_error'] + 3 * self.train_stats['std_error'])
        
        # Переопределяем маску на основе конкретного порога запроса, если нужно
        # Здесь используем комбинированный подход:
        final_threshold = np.percentile(full_scores, threshold_percentile)
        anomaly_mask = full_scores > final_threshold
        
        result = self.df.copy()
        result['anomaly_score'] = full_scores
        result['is_anomaly'] = anomaly_mask
        
        return result, final_threshold





    
    

In [18]:
# 2. Конфигурация
config = ModelConfig(
    window_size=50,
    embed_dim=128,
    hidden_dim=128,
    latent_dim=32,
    num_heads=8,
    num_layers=2,
    dropout=0.1,
    learning_rate=1e-3,
    graph_reg_lambda=1e-4, # Штраф за сложные графы
    device="auto"
)
    
print(f"Running on: {config.device}")
print(f"Data Shape: {df.shape}")

# 3. Инициализация и Обучение
detector = GraphAnomalyDetector(df, config)
    
print("\nTraining Model...")
detector.fit(epochs=100, batch_size=64, verbose=True)
    

Running on: auto
Data Shape: (2399, 53)

Training Model...
Epoch 10/100 | Loss: 0.9691 | Rec: 0.9585
Epoch 20/100 | Loss: 0.8205 | Rec: 0.8099
Epoch 30/100 | Loss: 0.8019 | Rec: 0.7913
Epoch 40/100 | Loss: 0.7568 | Rec: 0.7462
Epoch 50/100 | Loss: 0.7481 | Rec: 0.7375
Epoch 60/100 | Loss: 0.7687 | Rec: 0.7581
Epoch 70/100 | Loss: 0.7482 | Rec: 0.7376
Epoch 80/100 | Loss: 0.7435 | Rec: 0.7329
Epoch 90/100 | Loss: 0.7444 | Rec: 0.7338
Epoch 100/100 | Loss: 0.7427 | Rec: 0.7321


In [19]:
print("\nDetecting Anomalies...")
result_df, threshold = detector.detect(threshold_percentile=91.0)
    
print(f"\nThreshold: {threshold:.4f}")
anomalies = result_df[result_df['is_anomaly']]
print(f"Detected {len(anomalies)} anomalies.")
print("\nTop 5 Anomaly Scores:")
print(result_df.nlargest(5, 'anomaly_score'))


Detecting Anomalies...

Threshold: 3.0933
Detected 216 anomalies.

Top 5 Anomaly Scores:
       Ubs,V  Ibs,A  Isun,A  Ipt1,A  Ipt2,A  Ipt3,A  Ipt4,A  Ipt5,A  Ipt6,A  \
1955  11.029   4.51    3.81    0.69    3.22    0.76    3.20    3.38    4.73   
1956  11.037   4.51    3.81    3.18    3.22    3.24    3.20    3.38    4.73   
1954  11.020   4.51    3.81    3.18    3.22    3.22    3.20    3.38    4.73   
1957  11.055   4.51    3.81    3.18    3.22    3.24    3.20    3.38    4.73   
1958  11.090   4.51    3.81    3.18    3.29    3.24    3.22    3.38    4.73   

      Ipt7,A  ...  TNap,C  TPrd2,C  TPrd1,C  TDS24,C  power, W       D+  \
1955    3.76  ...    -128     -128        6      -23    49.740  357.084   
1956    3.76  ...    -128     -128        6       52    49.779  355.758   
1954    3.76  ...    -128     -128        6      -23    49.700  357.084   
1957    3.76  ...    -128     -128        6       52    49.858  355.758   
1958    3.76  ...    -128     -128        6       52    50.0

In [22]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score

print(confusion_matrix(df["Class"], result_df['is_anomaly']))
print(classification_report(df["Class"], result_df['is_anomaly']))


[[2076  113]
 [ 107  103]]
              precision    recall  f1-score   support

           0       0.95      0.95      0.95      2189
           1       0.48      0.49      0.48       210

    accuracy                           0.91      2399
   macro avg       0.71      0.72      0.72      2399
weighted avg       0.91      0.91      0.91      2399



In [23]:
print(df["Class"].sum(), result_df['is_anomaly'].sum())

210 216


In [34]:
d

,0,1,2,3,4,5,6,7,8,9,...,2389,2390,2391,2392,2393,2394,2395,2396,2397,2398
is_anomaly,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Class,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0


In [25]:
result_df.to_excel("graph_ae_res.xlsx")